<h1><center> CloudAEye MultiClass Classifier Part-1 </center></h1>

Connecting And Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/CloudAEye

/content/drive/My Drive/CloudAEye


Importing Libraries and Reading Dataframe


In [39]:
import pandas as pd
import numpy as np

from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost import XGBClassifier


In [4]:
df= pd.read_csv('consolidated-metrics.csv')
df.head()

,timestamp,apiserver_current_inflight_requests,apiserver_request_latencies_summary,apiserver_response_sizes_sum,apiserver_storage_transformation_latencies_microseconds_sum,aws_cpu_utilization,aws_ec2_cpuutilization_average,aws_ec2_network_out_average,aws_network_out,cgroup_manager_operation_rate,container_cpu_cfs_throttled_seconds_total,container_fs_inodes_free,container_fs_usage_bytes,container_fs_writes_total,container_memory_cache,container_memory_mapped_file,container_memory_rss,container_memory_usage_bytes,container_memory_working_set_bytes,coredns_kubernetes_dns_programming_duration_seconds_count,coredns_kubernetes_dns_programming_duration_seconds_sum,etcd_object_counts,etcd_request_latencies_summary_count,http_request_duration_microseconds,http_response_size_bytes,httpsessions_active,kafka_log_size_growth_rate,kafka_network_receive,kafka_network_transmit,kanban_boards_creation,kanban_tasks_complete_with_200_status,kanban_tasks_get_history,kube_pod_start_time,kube_poddisruptionbudget_status_current_healthy,kube_replicaset_created,kubelet_cgroup_manager_latency_microseconds,kubelet_container_log_filesystem_used_bytes,kubelet_operation_rate,kubelet_pleg_relist_interval_microseconds,kubelet_pleg_relist_latency_microseconds,...,node_filesystem_free_bytes,node_load1,node_load15,node_load5,node_memory_Active_anon_bytes,node_memory_Active_bytes,node_memory_Active_file_bytes,node_memory_AnonHugePages_bytes,node_memory_AnonPages_bytes,node_memory_Buffers_bytes,node_memory_Cached_bytes,node_memory_Committed_AS_bytes,node_memory_Dirty_bytes,node_memory_Inactive_anon_bytes,node_memory_Inactive_bytes,node_memory_Inactive_file_bytes,node_memory_KernelStack_bytes,node_memory_Mapped_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_SReclaimable_bytes,node_memory_SUnreclaim_bytes,node_memory_Shmem_bytes,node_memory_Slab_bytes,node_memory_Writeback_bytes,node_netstat_Tcp_CurrEstab,node_nf_conntrack_entries,node_procs_blocked,node_procs_running,node_scrape_collector_duration_seconds,node_sockstat_TCP_inuse,node_sockstat_TCP_mem,node_sockstat_TCP_mem_bytes,node_sockstat_TCP_orphan,node_sockstat_TCP_tw,node_sockstat_sockets_used,node_timex_estimated_error_seconds,node_timex_frequency_adjustment_ratio,node_timex_maxerror_seconds,status
0,2020-09-18 21:45:03,0.008333,4.775001e+06,6871.300000,0.025000,13.8,0.000,0.000000,0.000000,0.145833,0.0,155876.292281,4.582444e+08,31.947430,3.290259e+07,6.833672e+06,2.183418e+08,2.930822e+08,2.824967e+08,0.0,0.0,0.308333,7.333333,6.347728,0.066667,0.375000,1220.783333,5724.919214,8424.376927,0.0,0.3750,0.3750,0.0,0.0,0.0,2.716667,1.131008e+06,0.0,84978.795216,1549.886504,...,2.470222e+08,0.017197,0.018250,0.014958,5.852687e+07,6.471299e+07,5.355418e+06,0.0,7.410060e+07,153.600000,9.880303e+06,2.244632e+08,11963.733333,1757.866667,3.394628e+06,3.392870e+06,292044.800000,4.656657e+06,4.891639e+07,1.323989e+07,1.310447e+06,3.931375e+06,6553.600000,7.881847e+06,0.000000,0.033333,15.125000,0.0,0.120833,0.001250,0.054167,2.233333,9147.733333,0.000000,0.983333,25.441667,7.625000e-07,0.025000,0.000291,NORMAL
1,2020-09-18 21:46:03,0.008333,3.862502e+06,7065.713485,0.025000,19.2,0.025,1752.173542,1752.173542,0.145833,0.0,112788.516331,4.584697e+08,26.978072,1.859325e+07,6.970226e+06,2.306738e+08,3.136550e+08,3.100925e+08,0.0,0.0,0.608335,7.358352,186.125180,0.091667,13.237500,752.341667,2623.485430,4151.707333,0.0,0.2250,0.2250,0.0,0.0,0.0,2.016667,1.505963e+06,0.0,76434.145833,1424.675000,...,2.866220e+08,0.018625,0.018750,0.014417,7.601149e+07,8.622403e+07,5.355025e+06,0.0,8.548780e+07,170.666667,9.875763e+06,2.561419e+08,12834.133333,1757.866667,3.390208e+06,3.388450e+06,391031.466667,6.656427e+06,5.041085e+07,1.104294e+07,1.823659e+06,5.218253e+06,12117.333333,1.042432e+07,0.000000,0.029167,13.591667,0.0,0.108333,0.001182,0.050000,2.887500,11827.200000,0.000000,0.666667,25.420833,7.458333e-07,0.025000,0.000365,CPU_BURN_KANBAN_API_GATEWAY
2,2020-09-18 21:47:03,0.008333,3.862502e+06,6675.4

In [5]:
df.shape

(4294, 97)

In [6]:
df['status'].value_counts()

NORMAL                                  2757
NETWORK_DELAY_KANBAN_API_GATEWAY         646
NETWORK_DELAY_KANBAN_COMMAND_SERVICE     500
POD_KILL_KANBAN_API_GATEWAY              310
CPU_BURN_KANBAN_API_GATEWAY               80
Name: status, dtype: int64

Taking a look at Missing Data

In [7]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)

In [15]:
missing_value_df.head(20)

,column_name,percent_missing
kubelet_pod_worker_duration_bucket,kubelet_pod_worker_duration_bucket,94.853284
kanban_tasks_get_history,kanban_tasks_get_history,4.913833
kanban_tasks_complete_with_200_status,kanban_tasks_complete_with_200_status,4.913833
kafka_log_size_growth_rate,kafka_log_size_growth_rate,0.419190
kubelet_volume_stats_used_bytes,kubelet_volume_stats_used_bytes,0.116442
kubelet_volume_stats_inodes_used,kubelet_volume_stats_inodes_used,0.116442
kubelet_volume_stats_inodes_free,kubelet_volume_stats_inodes_free,0.116442
kubelet_volume_stats_available_bytes,kubelet_volume_stats_available_bytes,0.116442
kafka_network_transmit,kafka_network_transmit,0.093153
kafka_network_receive,kafka_network_receive,0.093153


In [9]:
tempdf1 = df[df['mysql_bytes_received'].isna()]
tempdf1['status'].unique()

array(['NORMAL', nan], dtype=object)

There is only one column named 'kubelet_pod_worker_duration_bucket' which has more than 94%missing data. Therefore removing this column.

Dropping column with high missing data

In [10]:
columns_to_remove  = list(missing_value_df[missing_value_df.percent_missing >50].column_name)
df.drop(columns_to_remove,axis=1,inplace=True)

In [11]:
df.shape

(4294, 96)

Also, we have some missing values in Target variable. Therfore removing those

In [18]:
df = df[df['status'].notna()]

In [19]:
df.shape

(4293, 96)

In [20]:
df['status'].unique()

array(['NORMAL', 'CPU_BURN_KANBAN_API_GATEWAY',
       'NETWORK_DELAY_KANBAN_API_GATEWAY',
       'NETWORK_DELAY_KANBAN_COMMAND_SERVICE',
       'POD_KILL_KANBAN_API_GATEWAY'], dtype=object)

Taking a look at columns that are not integer or float type

In [22]:
for i in df.columns:
  if df[i].dtype != np.number:
    print(i)

timestamp
status


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  


# I will be using XGboost here due to reasons as below
## Since there is Missing data in few column. Xgboost will be able to handle it internally.
## No need to scale or transform the data.
## Since this is an ensemble model.Correlation will not be an issue here 

## Preparing Data ForXGBOOST

In [28]:
df_xg = df.iloc[:,1:].copy()
df_xg.head()

,apiserver_current_inflight_requests,apiserver_request_latencies_summary,apiserver_response_sizes_sum,apiserver_storage_transformation_latencies_microseconds_sum,aws_cpu_utilization,aws_ec2_cpuutilization_average,aws_ec2_network_out_average,aws_network_out,cgroup_manager_operation_rate,container_cpu_cfs_throttled_seconds_total,container_fs_inodes_free,container_fs_usage_bytes,container_fs_writes_total,container_memory_cache,container_memory_mapped_file,container_memory_rss,container_memory_usage_bytes,container_memory_working_set_bytes,coredns_kubernetes_dns_programming_duration_seconds_count,coredns_kubernetes_dns_programming_duration_seconds_sum,etcd_object_counts,etcd_request_latencies_summary_count,http_request_duration_microseconds,http_response_size_bytes,httpsessions_active,kafka_log_size_growth_rate,kafka_network_receive,kafka_network_transmit,kanban_boards_creation,kanban_tasks_complete_with_200_status,kanban_tasks_get_history,kube_pod_start_time,kube_poddisruptionbudget_status_current_healthy,kube_replicaset_created,kubelet_cgroup_manager_latency_microseconds,kubelet_container_log_filesystem_used_bytes,kubelet_operation_rate,kubelet_pleg_relist_interval_microseconds,kubelet_pleg_relist_latency_microseconds,kubelet_pod_start_rate,...,node_filesystem_free_bytes,node_load1,node_load15,node_load5,node_memory_Active_anon_bytes,node_memory_Active_bytes,node_memory_Active_file_bytes,node_memory_AnonHugePages_bytes,node_memory_AnonPages_bytes,node_memory_Buffers_bytes,node_memory_Cached_bytes,node_memory_Committed_AS_bytes,node_memory_Dirty_bytes,node_memory_Inactive_anon_bytes,node_memory_Inactive_bytes,node_memory_Inactive_file_bytes,node_memory_KernelStack_bytes,node_memory_Mapped_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_SReclaimable_bytes,node_memory_SUnreclaim_bytes,node_memory_Shmem_bytes,node_memory_Slab_bytes,node_memory_Writeback_bytes,node_netstat_Tcp_CurrEstab,node_nf_conntrack_entries,node_procs_blocked,node_procs_running,node_scrape_collector_duration_seconds,node_sockstat_TCP_inuse,node_sockstat_TCP_mem,node_sockstat_TCP_mem_bytes,node_sockstat_TCP_orphan,node_sockstat_TCP_tw,node_sockstat_sockets_used,node_timex_estimated_error_seconds,node_timex_frequency_adjustment_ratio,node_timex_maxerror_seconds,status
0,0.008333,4.775001e+06,6871.300000,0.025000,13.8,0.000,0.000000,0.000000,0.145833,0.0,155876.292281,4.582444e+08,31.947430,3.290259e+07,6.833672e+06,2.183418e+08,2.930822e+08,2.824967e+08,0.0,0.0,0.308333,7.333333,6.347728,0.066667,0.375000,1220.783333,5724.919214,8424.376927,0.0,0.3750,0.3750,0.0,0.0,0.0,2.716667,1.131008e+06,0.0,84978.795216,1549.886504,0.0,...,2.470222e+08,0.017197,0.018250,0.014958,5.852687e+07,6.471299e+07,5.355418e+06,0.0,7.410060e+07,153.600000,9.880303e+06,2.244632e+08,11963.733333,1757.866667,3.394628e+06,3.392870e+06,292044.800000,4.656657e+06,4.891639e+07,1.323989e+07,1.310447e+06,3.931375e+06,6553.600000,7.881847e+06,0.000000,0.033333,15.125000,0.0,0.120833,0.001250,0.054167,2.233333,9147.733333,0.000000,0.983333,25.441667,7.625000e-07,0.025000,0.000291,NORMAL
1,0.008333,3.862502e+06,7065.713485,0.025000,19.2,0.025,1752.173542,1752.173542,0.145833,0.0,112788.516331,4.584697e+08,26.978072,1.859325e+07,6.970226e+06,2.306738e+08,3.136550e+08,3.100925e+08,0.0,0.0,0.608335,7.358352,186.125180,0.091667,13.237500,752.341667,2623.485430,4151.707333,0.0,0.2250,0.2250,0.0,0.0,0.0,2.016667,1.505963e+06,0.0,76434.145833,1424.675000,0.0,...,2.866220e+08,0.018625,0.018750,0.014417,7.601149e+07,8.622403e+07,5.355025e+06,0.0,8.548780e+07,170.666667,9.875763e+06,2.561419e+08,12834.133333,1757.866667,3.390208e+06,3.388450e+06,391031.466667,6.656427e+06,5.041085e+07,1.104294e+07,1.823659e+06,5.218253e+06,12117.333333,1.042432e+07,0.000000,0.029167,13.591667,0.0,0.108333,0.001182,0.050000,2.887500,11827.200000,0.000000,0.666667,25.420833,7.458333e-07,0.025000,0.000365,CPU_BURN_KANBAN_API_GATEWAY
2,0.008333,3.862502e+06,6675.475000,0.025000,19.2,0.050,3504.347083,3

In [30]:
missing_indicator = -1000
df_xg.fillna(missing_indicator, inplace=True)

Using label encoder for the Target variable 'Status'

In [33]:
# encode string class values as integers


label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(df_xg['status'])
label_encoded_y = label_encoder.transform(df_xg['status'])

In [34]:
df_xg.drop('status',axis=1,inplace=True)

In [36]:
seed = 7
test_size = 0.20
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_xg, label_encoded_y, test_size=test_size, random_state=seed)

Creating XGboost Dmatrix

In [38]:
# convert data to xgb data structure
xgtrain = xgb.DMatrix(X_train, y_train, 
                        missing=missing_indicator)
xgtest = xgb.DMatrix(X_test, label=y_test, 
                        missing=missing_indicator)

In [43]:
xgb_num_rounds = 720
def get_params():
    
    params = {}
    params["objective"] = "reg:linear"     
    params["eta"] = 0.05
    params["min_child_weight"] = 360
    params["subsample"] = 0.85
    params["colsample_bytree"] = 0.3
    params["silent"] = 1
    params["max_depth"] = 7
    plst = list(params.items())

    return plst

In [47]:
def eval_wrapper(yhat, y):  
    y = np.array(y)
    y = y.astype(int)
    yhat = np.array(yhat)
    yhat = np.clip(np.round(yhat), np.min(y), np.max(y)).astype(int)   
    accuracy_score(y, yhat)

In [42]:
# get the parameters for xgboost
plst = get_params()
print(plst) 

[('objective', 'reg:linear'), ('eta', 0.05), ('min_child_weight', 360), ('subsample', 0.85), ('colsample_bytree', 0.3), ('silent', 1), ('max_depth', 7)]


Training the model

In [44]:
# train model
model = xgb.train(plst, xgtrain, xgb_num_rounds) 

In [62]:
# get preds
train_preds = model.predict(xgtrain, ntree_limit=model.best_iteration)
tp_round = train_preds.round().astype(int)
print('Train score is:', eval_wrapper(tp_round, y_train)) 
test_preds = model.predict(xgtest, ntree_limit=model.best_iteration)

Train score is: None


In [64]:
test_prediction = test_preds.round().astype(int)

Predicting Scores

In [63]:
accuracy_score(y_train, tp_round)

0.8948747815958067

In [65]:
accuracy_score(y_test, test_prediction)


0.8323632130384168

## Since i dont have enough infromation about this data and its importance. I used accuracy as the metric for judgement.
Based on the results the test accuracy is over **83%** and train accuracy is over **89%**